### Import libraries



In [2]:
# for load dataset
from google.colab import drive
import sys
from pathlib import Path
# for data processing
import numpy as np # linear algebra
import pandas as pd
# for train data split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from keras import layers
from keras.models import load_model
from tensorflow import keras
tf.random.set_seed(363)

### Load Dataset & Model

Download Pre-trained Model

In [19]:
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
model = keras.models.load_model("/content/drive/MyDrive/ML_Final/0816016_weight.h5")

Load Test Dataset

In [5]:
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [6]:
base = Path('/gdrive/MyDrive/ML_Final/')
sys.path.append(str(base))

In [7]:
zip_path = base/'tabular.zip'
!cp "{zip_path}" .
!unzip -q tabular.zip
!rm tabular.zip

In [8]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

### Inference

Data Preprocess

In [9]:
# Replace missing columns with incomplete values in dataset

null_cols_number = []
for i in test_df.columns:
  if test_df[i].isna().value_counts()[0]-len(test_df[i]) < 0:
    null_cols_number.append(i)

for col in null_cols_number:
  null_cols = test_df[test_df[col].isna()].index
  for n in null_cols:
    test_df.loc[n,col] = test_df[col].mean()

In [10]:
test_df['product_code'] = test_df['product_code'].factorize()[0]
test_df['attribute_0'] = test_df['attribute_0'].str.lstrip('material_').astype('int')
test_df['attribute_1'] = test_df['attribute_1'].str.lstrip('material_').astype('int')

In [11]:
total_cols = list(train_df.columns)
# Removing strings columns
total_cols.remove("id")
total_cols.remove("product_code")
total_cols.remove("attribute_0")
total_cols.remove("attribute_1")
total_cols.remove("attribute_2")

One-hot encoding

In [32]:
test_dum = pd.get_dummies(test_df[total_cols[:-1]],drop_first=True)

Predict Test Data

In [46]:
y_pred = list(model.predict(test_dum))

650/650 [==============================] - 1s 970us/step


In [47]:
print(y_pred, '\n')

[array([0.19616008], dtype=float32), array([0.1874158], dtype=float32), array([0.19042382], dtype=float32), array([0.18935505], dtype=float32), array([0.29488492], dtype=float32), array([0.16085224], dtype=float32), array([0.16681722], dtype=float32), array([0.22468801], dtype=float32), array([0.15375714], dtype=float32), array([0.16837987], dtype=float32), array([0.1547526], dtype=float32), array([0.20167096], dtype=float32), array([0.18625914], dtype=float32), array([0.17593044], dtype=float32), array([0.22276819], dtype=float32), array([0.16774254], dtype=float32), array([0.17237115], dtype=float32), array([0.16599411], dtype=float32), array([0.21449088], dtype=float32), array([0.18594041], dtype=float32), array([0.19979115], dtype=float32), array([0.2245969], dtype=float32), array([0.22372036], dtype=float32), array([0.18838157], dtype=float32), array([0.16883019], dtype=float32), array([0.22058077], dtype=float32), array([0.22315192], dtype=float32), array([0.1829314], dtype=float

In [38]:
print(len(y_pred))

20775


Produce submission file

In [48]:
# Final changes
test_df["failure"] = y_pred
test_df["failure"] = test_df["failure"].astype("float64")
Final_prediction = test_df[["id","failure"]]
# CSV
Final_prediction.to_csv("0816016.csv",index=False)